In [1]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=q3kynzoFkYgAbqoqKdtmS3Xlj526HU&access_type=offline&code_challenge=fjl9Djkph130HrJBo8-772hhU3FrNWMVO4ObV-ICYqk&code_challenge_method=S256


You are now logged in as [yassine.babdallah0@gmail.com].
Your current project is [projet-genai].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [2]:
%pip install -q pandas google-cloud-storage langchain-google-vertexai langchain-google-cloud-sql-pg python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from google.cloud import storage

# Initialize a client
client = storage.Client()

# Access the bucket
bucket_name = 'medical-bucket12'
bucket = client.bucket(bucket_name)

# List all files in the 'data' directory
blobs = bucket.list_blobs()

print("Files in 'dauphine-bucket/data':")
for blob in blobs:
    print(blob.name)

Files in 'dauphine-bucket/data':
medquad.csv


In [4]:
file_path = 'medquad.csv'

# Get the blob object
blob = bucket.get_blob(file_path)


if blob:
    print(f"Information for '{file_path}':")
    print(f"Size: {blob.size} bytes")
    print(f"Content Type: {blob.content_type}")
    updated_on = blob.updated
    formatted_updated_on = updated_on.strftime("%Y-%m-%d %H:%M:%S")  # Format: YYYY-MM-DD HH:MM:SS
    print(f"Updated On: {formatted_updated_on}")
    print(f"Blob name: {blob.name}")
else:
    print(f"File '{file_path}' not found in the bucket.")

Information for 'medquad.csv':
Size: 22835609 bytes
Content Type: text/csv
Updated On: 2025-02-11 19:19:34
Blob name: medquad.csv


In [5]:
import os
import pandas as pd
from google.cloud import storage
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_cloud_sql_pg import PostgresEngine, PostgresVectorStore
from langchain_core.documents.base import Document
from dotenv import load_dotenv
import asyncio

In [6]:
DOWNLOADED_LOCAL_DIRECTORY = "./downloaded_files"

# S'assurer que le dossier downloaded_files existe
os.makedirs(DOWNLOADED_LOCAL_DIRECTORY, exist_ok=True)

# Initialiser le client Storage
client = storage.Client()

# Obtenir le bucket
bucket = client.get_bucket(bucket_name)

# Télécharger le fichier
def download_file_from_bucket(bucket, file_path: str) -> str:
    blob = bucket.blob(file_path)
    local_file_name = os.path.basename(file_path)
    local_filepath = os.path.join(DOWNLOADED_LOCAL_DIRECTORY, local_file_name)
    blob.download_to_filename(local_filepath)
    print(f"Downloaded '{file_path}' to '{local_file_name}'")
    return local_filepath

In [8]:
def csv_to_documents(local_filepath: str) -> list[Document]:
    df = pd.read_csv(local_filepath)
    df['answer'] = df['answer'].fillna("Pas de réponse disponible")
    df['focus_area'] = df['focus_area'].fillna("Non spécifié")
    documents = []
    
    for index, row in df.iterrows():
        # Content contient uniquement la question
        content = row['question']
        
        # Metadata contient la réponse et les informations contextuelles
        metadata = {
            "answer": row['answer'],
            "source": row['source'],
            "focus_area": row['focus_area'],
            "row_index": index
        }
        
        doc = Document(page_content=content, metadata=metadata)
        documents.append(doc)
    
    return documents

In [9]:
file_path = "medquad.csv"  # Remplacez par votre nom de fichier
local_filepath = download_file_from_bucket(bucket, file_path)

# Utiliser directement le fichier local
local_filepath = "./downloaded_files/medquad.csv"


documents = csv_to_documents(local_filepath)

Downloaded 'medquad.csv' to 'medquad.csv'


In [10]:
print(f"Nombre de documents créés : {len(documents)}")
print("\nExemple du premier document :")
print(documents[0].page_content)
print("\nMétadonnées :", documents[0].metadata)

Nombre de documents créés : 16412

Exemple du premier document :
What is (are) Glaucoma ?

Métadonnées : {'answer': "Glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. While glaucoma can strike anyone, the risk is much greater for people over 60. How Glaucoma Develops  There are several different types of glaucoma. Most of these involve the drainage system within the eye. At the front of the eye there is a small space called the anterior chamber. A clear fluid flows through this chamber and bathes and nourishes the nearby tissues. (Watch the video to learn more about glaucoma. To enlarge the video, click the brackets in the lower right-hand corner. To reduce the video, press the Escape (Esc) button on your keyboard.) In glaucoma, for still unknown reasons, the fluid drains too slowly out of the eye. As the fluid builds up, the pressure inside the eye rises. Unless this pressure is controlled, it may cause damage to the optic n

In [11]:
%pip install --upgrade --quiet  langchain-google-cloud-sql-pg langchain-google-vertexai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
import os
from config import PROJECT_ID, REGION, INSTANCE, DATABASE, DB_USER, TABLE_NAME
DB_PASSWORD = os.environ["DB_PASSWORD"]

In [14]:
engine = PostgresEngine.from_instance(
    project_id=PROJECT_ID,
    instance=INSTANCE,
    region=REGION,
    database=DATABASE,
    user=DB_USER,
    password=DB_PASSWORD  
)

In [20]:
embedding = VertexAIEmbeddings(
    model_name='text-embedding-004',
    project=PROJECT_ID
)


In [19]:
# Create a table in the PostgreSQL database with the required columns
from sqlalchemy.exc import ProgrammingError

try:
    await engine.ainit_vectorstore_table(
        table_name=TABLE_NAME, # Vector size for VertexAI model(textembedding-gecko@latest)
        vector_size=768,
    )
except ProgrammingError:
    print("Table already created")

Table already created


In [21]:
vector_store = PostgresVectorStore.create_sync(  # Use .create() to initialize an async vector store
    engine=engine,
    table_name=TABLE_NAME,
    embedding_service=embedding,
)

In [22]:
vector_store.add_documents(documents)

['0678aade-6497-4052-8d6c-5eaadab67302',
 'a70aa8f2-c799-47b3-8385-fbc96418a463',
 'b4b5a994-b0df-414f-9e2c-aa3c83494f87',
 '0da6c99b-1afb-4ec8-8483-82b96d5dce35',
 'd9189095-49ff-49e3-835f-82323b519352',
 '6f2d95fc-1301-4091-aec6-c17665af2a26',
 'd63f14e4-f691-479d-9449-87560531f6cd',
 '176baefe-c4d1-4b12-acd5-b94f3389e514',
 '614b60ad-e155-4601-b091-ca53989246b5',
 '1f13f5b7-6757-425c-954b-0b25bdf72616',
 '21580a62-7bf2-43a7-b8f5-9e7244df9a51',
 '3fab07fe-73d3-4ad9-a34b-54802a4ca18d',
 'd4548ce9-257b-4a3a-8d84-f015c475ee11',
 '42868d05-9f4f-4d7a-a6b9-810696fbc6a4',
 '4c215dd9-c5e9-436d-9ced-98998b63a1e0',
 'b23adb24-ced4-4c96-b935-632c273bcc40',
 '04b682dc-ef61-4e44-9581-110521dcfb27',
 '99aa1a08-b7b9-4e66-ada1-6c6fa92c6683',
 '7f3018fc-ef6d-46f3-a338-a044f84614e1',
 'd4fc1979-7822-4160-ba07-4d3f272c24c4',
 '9289427b-91bf-4031-a181-8f1cd082b22f',
 '4a2315a2-5f6d-4381-8208-d03d1e62e341',
 '7bc94402-7bfb-4869-9ef2-a7c101c50193',
 '44383526-3e59-4123-b617-2c698dc1926d',
 '2da42720-59e7-